In [3]:
import pandas as pd
import numpy as np
import requests
import datetime
import plotly.graph_objs as go
import plotly.express as px
import locale

In [165]:
# SKIP


def load_chain_data(chain_id):
    chain_url = 'https://indexer-grants-stack.gitcoin.co/data/' + chain_id + '/rounds.json'
    try:
        response = requests.get(chain_url)
        if response.status_code == 200:
            chain_data = response.json()
            rounds = []
            for round in chain_data:
                if round['metadata'] is not None:
                    round_data = {
                        'round_id': round['id'],
                        'name': round['metadata']['name'],
                        'amountUSD': round['amountUSD'],
                        'votes': round['votes'],
                        'description': round['metadata']['description'] if 'description' in round['metadata'] else '',
                        'matchingFundsAvailable': round['metadata']['matchingFunds']['matchingFundsAvailable'] if 'matchingFunds' in round['metadata'] else '',
                        'matchingCap': round['metadata']['matchingFunds']['matchingCap'] if 'matchingFunds' in round['metadata'] else '',
                        'roundStartTime': datetime.datetime.utcfromtimestamp(int(round['roundStartTime'])), # create a datetime object from the timestamp in UTC time
                        'roundEndTime': datetime.datetime.utcfromtimestamp(int(round['roundEndTime']))
                    }
                    rounds.append(round_data)
            df = pd.DataFrame(rounds)
            #start_time = datetime.datetime(2023, 4, 26, 15, 0, 0)
            #end_time = datetime.datetime(2023, 5, 9, 23, 59, 0)
            df = df[(df['votes'] > 0)] #& (df['roundStartTime'] <= start_time) & (df['roundEndTime'] == end_time)]
            return df 
    except: 
        return pd.DataFrame()
    
def load_round_projects_data(round_id):
    # prepare the URLs
    projects_url = 'https://indexer-grants-stack.gitcoin.co/data/1/rounds/' + round_id + '/projects.json'
    
    try:
        # download the Projects JSON data from the URL
        response = requests.get(projects_url)
        if response.status_code == 200:
            projects_data = response.json()

        # Extract the relevant data from each project
        projects = []
        for project in projects_data:
            project_data = {
                'id': project['id'],
                'title': project['metadata']['application']['project']['title'],
                'description': project['metadata']['application']['project']['description'],
                'status': project['status'],
                'amountUSD': project['amountUSD'],
                'votes': project['votes'],
                'uniqueContributors': project['uniqueContributors']
            }
            projects.append(project_data)
        # Create a DataFrame from the extracted data
        dfp = pd.DataFrame(projects)
        # Reorder the columns to match the desired order and rename column id to project_id
        dfp = dfp[['id', 'title', 'description', 'status', 'amountUSD', 'votes', 'uniqueContributors']]
        dfp = dfp.rename(columns={'id': 'project_id'})
        # Filter to only approved projects
        #dfp = dfp[dfp['status'] == 'APPROVED']
        return dfp
    except:
        return pd.DataFrame()
    
def load_round_votes_data(round_id):
    votes_url = 'https://indexer-grants-stack.gitcoin.co/data/1/rounds/' + round_id + '/votes.json'
    try:
        # download the Votes JSON data from the URL
        response = requests.get(votes_url)
        if response.status_code == 200:
            votes_data = response.json()
        df = pd.DataFrame(votes_data)
        return df
    except:
        return pd.DataFrame()
    

def load_all_votes_data(chain_data):
    dfv_all = pd.DataFrame()
    for round_id in chain_data['round_id']:
        if round_id == '0x0000000000000000000000000000000000000000':
            print("*******DUDE Skipping round 0")
            continue
        #print("Loading Round with ID: " + round_id)
        dfv = load_round_votes_data(round_id)
        dfp = load_round_projects_data(round_id)
        dfv = pd.merge(dfv, dfp[['project_id', 'title', 'status']], how='left', left_on='projectId', right_on='project_id')
        # CHECK IF chain_data has columns roun_type and matching_amount
        if 'round_type' not in chain_data.columns:
            chain_data['round_type'] = ''
        if 'matching_amount' not in chain_data.columns:
            chain_data['matching_amount'] = 0
        if 'round_name' not in chain_data.columns:
            #rename name column to round_name
            chain_data = chain_data.rename(columns={'name': 'round_name'})
        dfv = pd.merge(dfv, chain_data[['round_id', 'round_name', 'round_type', 'matching_amount']], how='left', left_on='roundId', right_on='round_id')
        dfv_all = pd.concat([dfv_all, dfv])
    return dfv_all

def load_passport_data():
    url = 'https://indexer-grants-stack.gitcoin.co/data/passport_scores.json'
    try:
        response = requests.get(url)
        response.raise_for_status()
        passports_data = response.json()
        if not passports_data:
            raise ValueError('Passport data is empty')
        else:
            print("Passport data loaded successfully!")
        passports = []
        for passport in passports_data:
            data = {
                'address': passport.get('address', ''),
                'score': passport.get('score', ''),
                'status': passport.get('status', ''),
                'last_score_timestamp': passport.get('last_score_timestamp', ''),
                'evidence_type': passport['evidence'].get('type', '') if passport.get('evidence') else '',
                'evidence_success': passport['evidence'].get('success', '') if passport.get('evidence') else '',
                'evidence_rawScore': passport['evidence'].get('rawScore', '') if passport.get('evidence') else '',
                'evidence_threshold': passport['evidence'].get('threshold', '') if passport.get('evidence') else '',
                'error': passport.get('error', ''),
            }
            passports.append(data)
        df = pd.DataFrame(passports)
        return df
    except Exception as e:
        print("Error:", e)
        return pd.DataFrame()

In [3]:
# SKIP 
chain_id = '1'
df = load_chain_data(chain_id)
df_all = load_all_votes_data(df)
display(df_all)
# save df_all to a csv
df_all.to_csv('beta_votes_all.csv', index=False)
df = load_passport_data()
# Save to CSV
df.to_csv('passport_data.csv', index=False)

Loading Round with ID: 0x12BB5bBbFE596dbc489d209299B8302c3300fa40
Loading Round with ID: 0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094
Loading Round with ID: 0x274554EB289004e15A7679123901B7F070dDa0fa
Loading Round with ID: 0xAA40E2E5c8df03d792A52B5458959C320F86ca18


KeyError: "None of [Index(['project_id', 'title', 'status'], dtype='object')] are in the [columns]"

In [4]:
# read dfs from csvs
## START HERE
df_all = pd.read_csv('beta_votes_all.csv')
df_pp = pd.read_csv('passport_data.csv')


In [29]:
display(load_chain_data(chain_id))

,round_id,name,amountUSD,votes,description,matchingFundsAvailable,matchingCap,roundStartTime,roundEndTime
5,0x350E2257548dF38C6b2009245332d73f5919834e,testing testing 123,1.13,2,,10000,False,2023-01-05 22:51:00,2023-01-08 00:00:00
6,0xD95A1969c41112cEE9A2c931E849bCef36a16F4C,Web3 Open Source Software Round,502550.02,161129,,333333.57,True,2023-01-17 12:00:00,2023-01-31 23:59:00
7,0x1b165fE4DA6bC58AB8370DDC763d367D29F50Ef0,Climate Solutions Round,76855.60,14515,,333333.56,True,2023-01-17 12:00:00,2023-01-31 23:59:00
8,0xe575282b376E3c9886779A841A2510F1Dd8C2CE4,Ethereum Infrastructure Round,94880.05,20967,,333333.56,True,2023-01-17 12:00:00,2023-01-31 23:59:00
12,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,Web3 Open Source Software Round,330870.67,81811,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
13,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,Ethereum Infrastructure,41960.92,4307,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
15,0x274554EB289004e15A7679123901B7F070dDa0fa,ZK Tech Round,31238.92,8605,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
16,0xAA40E2E5c8df03d792A52B5458959C320F86ca18,Web3 Community and Education,62567.68,2896,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
17,0x421510312C40486965767be5Ea603Aa8a5707983,Climate Solutions,40646.31,1970,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
18,0x8aA06b3b8cAc2970857F4E0fD78F21dc01AAdE94,Metacrisis,17754.87,600,,,,2023-04-25 12:00:00,2023-05-09 23:59:00


In [11]:
### ADJUST PARAMETERS HERE
min_passport_score = 15.0
min_amountUSD = .90
power = 2 # change to 2
matching_cap_ratio = 0.10
matching_pool = 350000


def get_matching(votes_data):
    votes_data = get_voting_weights(votes_data)
    votes_data = votes_data[votes_data['voting_success'] == True]
    project_data = votes_data.groupby(['title']).agg({'amountUSD': 'sum', 'sqrt_amountUSD':'sum', 'id': 'sum', 'voter': 'nunique'}).reset_index().sort_values('amountUSD', ascending=False)
    project_data = project_data.rename(columns={'sqrt_amountUSD': 'sum_sqrt_amountUSD'})
    project_data['squared_sum_sqrts'] = project_data['sum_sqrt_amountUSD'] ** power
    project_data['squared_sum_sqrts_minus_donations'] = project_data['squared_sum_sqrts'] - project_data['amountUSD'] #minus green squares
    project_data['matching_ratio'] = project_data['squared_sum_sqrts_minus_donations'] / project_data['squared_sum_sqrts_minus_donations'].sum()
    project_data['matching_amount'] = project_data['matching_ratio'] * matching_pool
    if (project_data['matching_ratio'] > matching_cap_ratio).any():
        project_data = check_matching_cap(project_data)
    round_saturation = (project_data['squared_sum_sqrts_minus_donations'].sum() / matching_pool) * 100
    print("Round Saturation: " + '{:,.2f}'.format(round_saturation) + "%") 
    return project_data

def get_voting_weights(votes_data):
    votes_data_with_weights = votes_data#.groupby(['voter', 'title']).agg({'amountUSD': 'sum', 'id': 'nunique'}).reset_index().sort_values('amountUSD', ascending=False)
    # flag where min_amount_usd threshold not met 
    votes_data_with_weights['amountUSD_above_min'] = np.where(votes_data_with_weights['amountUSD'] > min_amountUSD, True, False)
    votes_data_with_weights['passport_score'] = votes_data_with_weights['voter'].str.lower().map(df_pp.set_index('address')['evidence_rawScore'])
    votes_data_with_weights['passport_success'] = np.where(votes_data_with_weights['passport_score'] >= min_passport_score, True, False)
    votes_data_with_weights['voting_success'] = (votes_data_with_weights['amountUSD_above_min'] & votes_data_with_weights['passport_success'])
    votes_data_with_weights['sqrt_amountUSD'] = (votes_data_with_weights['amountUSD']) ** (1/power)
    return votes_data_with_weights

def check_matching_cap(project_data):
    matching_cap_amount = matching_cap_ratio * matching_pool
    while True:
        # Calculate over cap amounts and total uncapped fund
        over_cap = np.maximum(0, project_data['matching_amount'] - matching_cap_amount)
        total_fund_for_not_capped = project_data['matching_amount'][project_data['matching_amount'] < matching_cap_amount].sum()

        # Reduce matching amounts that are over cap
        project_data.loc[project_data['matching_amount'] > matching_cap_amount, 'matching_amount'] = matching_cap_amount
               # Calculate remainder percent and update matching amounts
        if total_fund_for_not_capped > 0:
            remainder_percent = over_cap.sum() / total_fund_for_not_capped
            project_data.loc[project_data['matching_amount'] < matching_cap_amount, 'matching_amount'] *= (1 + remainder_percent)
        else:
            break  # Exit loop when there's no fund left for not capped projects
        
        # Ensure that updates didn't push any projects over the cap
        over_cap_after_update = np.maximum(0, project_data['matching_amount'] - matching_cap_amount)
        if not over_cap_after_update.sum() > 0:
            break  # Exit loop when there's no amount over cap
    project_data['matching_ratio'] = project_data['matching_amount'] / matching_pool
    return project_data

In [9]:
# filter df_all to only include round_id that is 0x274554EB289004e15A7679123901B7F070dDa0fa
df_zk = df_all[df_all['round_id'] == '0x421510312C40486965767be5Ea603Aa8a5707983']
#df_test = get_voting_weights(df_zk)
# save to csv
#df_test.to_csv('beta_votes_oss_new.csv', index=False)
df_test = get_matching(df_zk)
df_test = df_test.sort_values('matching_ratio', ascending=False)
# display without scientific notation 
pd.options.display.float_format = '{:.2f}'.format
# print dataframe matching_amount and matching_ratio sums
print("matching_amount sum: " + '{:,.2f}'.format(df_test['matching_amount'].sum()))
print("matching_ratio sum: " + '{:,.2f}'.format(df_test['matching_ratio'].sum()))
#display(df_test)
# filter to title, matching_amount, matching_ratio
df_test = df_test[['title', 'matching_amount', 'matching_ratio']]
# display all rows
pd.set_option('display.max_rows', None)
# display dataframe
display(df_test.reset_index(drop=True))

Round Saturation: 116.51%
matching_amount sum: 350,000.00
matching_ratio sum: 1.00


/var/folders/vc/4gnlrkhx1bqgc_p_rhn78pxh0000gn/T/ipykernel_871/883549841.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votes_data_with_weights['amountUSD_above_min'] = np.where(votes_data_with_weights['amountUSD'] > min_amountUSD, True, False)
/var/folders/vc/4gnlrkhx1bqgc_p_rhn78pxh0000gn/T/ipykernel_871/883549841.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votes_data_with_weights['passport_score'] = votes_data_with_weights['voter'].str.lower().map(df_pp.set_index('address')['evidence_rawSc

,title,matching_amount,matching_ratio
0,Atlantis,35000.00,0.10
1,WarmOFF - CO₂ removal project,35000.00,0.10
2,$Earth - Solarpunk Dao,32091.31,0.09
3,Block2030,25267.45,0.07
4,Treegens - The Most Transparent & Rewarding Tr...,24039.83,0.07
5,Mini Meadows,22453.27,0.06
6,diosdao.xyz by Mycelia : Indigenous Public Goo...,22069.17,0.06
7,Treejer Protocol,20397.04,0.06
8,WaterDAO,10146.52,0.03
9,Web3beach,9934.32,0.03


In [12]:
# filter df_all to only include round_id that is 0x274554EB289004e15A7679123901B7F070dDa0fa
df_zk = df_all[df_all['round_id'] == '0x421510312C40486965767be5Ea603Aa8a5707983']
#df_test = get_voting_weights(df_zk)
# save to csv
#df_test.to_csv('beta_votes_oss_new.csv', index=False)
df_test = get_matching(df_zk)
df_test = df_test.sort_values('matching_ratio', ascending=False)
# display without scientific notation 
pd.options.display.float_format = '{:.2f}'.format
# print dataframe matching_amount and matching_ratio sums
print("matching_amount sum: " + '{:,.2f}'.format(df_test['matching_amount'].sum()))
print("matching_ratio sum: " + '{:,.2f}'.format(df_test['matching_ratio'].sum()))
#display(df_test)
# filter to title, matching_amount, matching_ratio
df_test = df_test[['title', 'matching_amount', 'matching_ratio']]
# display all rows
pd.set_option('display.max_rows', None)
# display dataframe
display(df_test.reset_index(drop=True))

Round Saturation: 123.01%
matching_amount sum: 350,000.00
matching_ratio sum: 1.00


/var/folders/vc/4gnlrkhx1bqgc_p_rhn78pxh0000gn/T/ipykernel_871/2497222026.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votes_data_with_weights['amountUSD_above_min'] = np.where(votes_data_with_weights['amountUSD'] > min_amountUSD, True, False)
/var/folders/vc/4gnlrkhx1bqgc_p_rhn78pxh0000gn/T/ipykernel_871/2497222026.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votes_data_with_weights['passport_score'] = votes_data_with_weights['voter'].str.lower().map(df_pp.set_index('address')['evidence_raw

,title,matching_amount,matching_ratio
0,WarmOFF - CO₂ removal project,35000.00,0.10
1,Atlantis,35000.00,0.10
2,$Earth - Solarpunk Dao,31492.39,0.09
3,Treegens - The Most Transparent & Rewarding Tr...,26071.08,0.07
4,Block2030,24074.07,0.07
5,diosdao.xyz by Mycelia : Indigenous Public Goo...,21577.05,0.06
6,Mini Meadows,21124.42,0.06
7,Treejer Protocol,19189.88,0.05
8,Web3beach,10257.95,0.03
9,WaterDAO,9546.02,0.03


In [47]:
# filter df_all to only include round_id that is 0x274554EB289004e15A7679123901B7F070dDa0fa
df_zk = df_all[df_all['round_id'] == '0x274554EB289004e15A7679123901B7F070dDa0fa']
matching_pool = 150000
#df_test = get_voting_weights(df_zk)
df_test = get_matching(df_zk, matching_pool)
df_test = df_test.sort_values('matching_ratio', ascending=False)
# display without scientific notation 
pd.options.display.float_format = '{:.2f}'.format
display(df_test)

Round Saturation: 9,099.72%


,title,amountUSD,sum_sqrt_amountUSD,id,voter,squared_sum_sqrts,squared_sum_sqrts_minus_donations,matching_ratio,matching_amount
10,zkBob,4163.12,1709.63,1016,993,2922822.00,2918658.88,0.21,32074.15
2,Halo2 Backend for Noir,3250.49,1413.00,865,847,1996579.06,1993328.56,0.15,21905.37
9,ZeroPool,2866.01,1281.23,827,807,1641558.28,1638692.27,0.12,18008.15
4,Proof of Innocence,3946.03,1225.45,774,756,1501726.42,1497780.39,0.11,16459.62
0,4EVERLAND,1969.24,1028.87,730,711,1058569.29,1056600.05,0.08,11611.34
5,Sprintcheckout,1839.15,990.63,705,688,981356.55,979517.41,0.07,10764.25
11,zkREPL,1822.92,975.58,692,675,951755.52,949932.60,0.07,10439.14
1,EZKL,2377.81,919.32,642,624,845141.38,842763.57,0.06,9261.42
3,PLUME: Pseudonymously Linked Unique Message En...,1770.33,887.79,637,624,788173.31,786402.98,0.06,8642.05
6,Switch Electric: Maxwell,1223.30,667.68,536,528,445794.41,444571.11,0.03,4885.55


In [5]:
display(df_all.head(5))

,id,transaction,blockNumber,projectId,applicationId,roundId,voter,grantAddress,token,amount,amountUSD,amountRoundToken,project_id,title,status,round_id,round_name,round_type,matching_amount
0,0xd9154a180dbccb2a854223eb1cdef145e96faa174616...,0x0328340819a513652d554fd0f558ac2eb360b57e126e...,17123199,0x60c037f9dfa5daee0f07bc26fb73393ef924e36c73c4...,18,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,0xa37d20cd446898508Ab961c6Be8d3fe1c15413fA,0x18aa467E40E1deFB1956708830A343c1D01d3D7C,0x0000000000000000000000000000000000000000,10000000000000000,18.368665,18380080590000000000,0x60c037f9dfa5daee0f07bc26fb73393ef924e36c73c4...,JediSwap,APPROVED,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,Web3 Open Source Software Round,NaN,0
1,0x28a2a4e9aa3c3a0853572c12b4f7fd948f5e4a9d2384...,0xba2f9df050c6cd43a2676eb64bc81df2e193ece3ceb7...,17123217,0xf05829c76e3da92a8942378240ab4dc70c8c509f558e...,13,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,0xa37d20cd446898508Ab961c6Be8d3fe1c15413fA,0xe126b3E5d052f1F575828f61fEBA4f4f2603652a,0x0000000000000000000000000000000000000000,10000000000000000,18.368665,18380080590000000000,0xf05829c76e3da92a8942378240ab4dc70c8c509f558e...,Revoke.cash - Helping you stay safe in web3,APPROVED,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,Web3 Open Source Software Round,NaN,0
2,0x4350cdd06449fcac96f5ceea26929efdf9d3f420fca6...,0x986bdbdc0430eed93518a84dce9d98aef7385c0a9237...,17123229,0xb6a332decc38c703130a537ce7fcfe5a24bd85e52e07...,6,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,0x0Cac11a7D29aB8C87D8fedB9Cc736146f4fe8E84,0x531Eb60b2dDE9a841BA358EE033533AAF90D1feA,0x0000000000000000000000000000000000000000,2000000000000000,3.673733,3676016110000000000,0xb6a332decc38c703130a537ce7fcfe5a24bd85e52e07...,IDriss - A more usable web3 for everyone,APPROVED,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,Web3 Open Source Software Round,NaN,0
3,0x0f8ed9cb2aa39a8b766637c0bf5e1860bd4557292ff9...,0x986bdbdc0430eed93518a84dce9d98aef7385c0a9237...,17123229,0xbe519f8c8f7ff2acb359dd757c5d2bf25b05d9fd29b8...,9,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,0x0Cac11a7D29aB8C87D8fedB9Cc736146f4fe8E84,0x3A5bd1E37b099aE3386D13947b6a90d97675e5e3,0x0000000000000000000000000000000000000000,2000000000000000,3.673733,3676016110000000000,0xbe519f8c8f7ff2acb359dd757c5d2bf25b05d9fd29b8...,Lenster,APPROVED,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,Web3 Open Source Software Round,NaN,0
4,0x7674f1341ac2bc442770d6a5fe3ff59bb89f2b44ed48...,0x986bdbdc0430eed93518a84dce9d98aef7385c0a9237...,17123229,0x60c037f9dfa5daee0f07bc26fb73393ef924e36c73c4...,18,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,0x0Cac11a7D29aB8C87D8fedB9Cc736146f4fe8E84,0x18aa467E40E1deFB1956708830A343c1D01d3D7C,0x0000000000000000000000000000000000000000,2000000000000000,3.673733,3676016110000000000,0x60c037f9dfa5daee0f07bc26fb73393ef924e36c73c4...,JediSwap,APPROVED,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,Web3 Open Source Software Round,NaN,0


In [7]:
## TEMPORARY STUFF
# add time information here 

self_votes = df_all[df_all['voter'] == df_all['grantAddress']]
self_votes = self_votes.groupby(['voter', 'title', 'grantAddress', 'round_name', 'round_id' ]).agg({'amountUSD': 'sum', 'id': 'nunique'}).reset_index().sort_values('id', ascending=False)

# save self_votes to a csv
self_votes.to_csv('self_votes.csv', index=False)

display(self_votes).head(10)

,voter,title,grantAddress,round_name,round_id,amountUSD,id
45,0xD0352D08AA93C9bc9C79d2B3a53846DCd7c9cb92,"ReFi Local Node: Cape Town, South Africa",0xD0352D08AA93C9bc9C79d2B3a53846DCd7c9cb92,ReFi Local Nodes,0x64aa545C9C63944f8e765d9a65eda3cbbDc6e620,16.470288,3
18,0x5683921269B041fbE66882dd6CADd48B7e5b5817,OmniBTC,0x5683921269B041fbE66882dd6CADd48B7e5b5817,Web3 Open Source Software Round,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,39.043027,2
31,0x9390fA8656A161442928b442300358D82bEC28b0,DeFinomics Labs,0x9390fA8656A161442928b442300358D82bEC28b0,Token Engineering,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,4.115243,1
33,0x9FFA790731d0406B553F6fbd418A4580ec3E0269,ETH Tricolor ETH Ecuador,0x9FFA790731d0406B553F6fbd418A4580ec3E0269,Web3 Community and Education,0xAA40E2E5c8df03d792A52B5458959C320F86ca18,5.968624,1
34,0xAa163c47065c22D17Ed1c47E3E244337D2056c17,JobStash,0xAa163c47065c22D17Ed1c47E3E244337D2056c17,Web3 Community and Education,0xAA40E2E5c8df03d792A52B5458959C320F86ca18,9.741872,1
35,0xB5F8bCEB2c1fcE8F6aFb7778445c97455FddB205,KI-LIFE²,0xB5F8bCEB2c1fcE8F6aFb7778445c97455FddB205,Metacrisis,0x8aA06b3b8cAc2970857F4E0fD78F21dc01AAdE94,7.067012,1
36,0xBBbb17A8C1Ce64be28faCd93b22450abB01B5DCA,ENSAA,0xBBbb17A8C1Ce64be28faCd93b22450abB01B5DCA,ENS Ecosystem,0x64E5b2228eF31437909900B38fC42Dd5E4B83147,18.863320,1
37,0xBBbb17A8C1Ce64be28faCd93b22450abB01B5DCA,ENSAA,0xBBbb17A8C1Ce64be28faCd93b22450abB01B5DCA,Global Chinese Community beta round,0x859FaeAa266BA13bd1E72eB6dd7A223902d1adFE,18.863320,1
38,0xBc7BeB2d44Eb0Cddf6c0F2560a21475c2E2aC19C,Just-DNA-Seq,0xBc7BeB2d44Eb0Cddf6c0F2560a21475c2E2aC19C,DeSci (Decentralized Science),0x6e8dC2e623204D61b0E59E668702654aE336c9f7,1313.576401,1
39,0xC19A8A7bcfF4bfaC344d95EF7146Db8fa7169370,Nextme,0xC19A8A7bcfF4bfaC344d95EF7146Db8fa7169370,ENS Ecosystem,0x64E5b2228eF31437909900B38fC42Dd5E4B83147,667.055353,1


AttributeError: 'NoneType' object has no attribute 'head'

In [50]:

# filter to grantAddress = 0xD165df4296C85e780509fa1eace0150d945d49Fd
display(df_all[df_all['grantAddress'] == '0xD165df4296C85e780509fa1eace0150d945d49Fd'].head(10))

,id,transaction,blockNumber,projectId,applicationId,roundId,voter,grantAddress,token,amount,amountUSD,amountRoundToken,project_id,title,status,round_id,round_name,round_type,matching_amount
81907,0x5b7126f9880b0fd3697cb46c49b83b06e95e644346e4...,0x16c610713965999e6a7e312c639e3f7bbbf209768a59...,17125453,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,8,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,0x9E47aA1666c03F211642B2A13e064307326fdd33,0xD165df4296C85e780509fa1eace0150d945d49Fd,0x0000000000000000000000000000000000000000,3000000000000000,5.61,5602369880000000000,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,EthStaker,APPROVED,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,Ethereum Infrastructure,NaN,0
81931,0x081c44019f084e95529f6d862bd0ada5f6a276bd949e...,0xf7f65168ae814099943bab8ee4a5340eeb5ef8c44273...,17125502,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,8,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,0x66C97dC031bB2f6aD421CeF7b01De0CA2AE4fe64,0xD165df4296C85e780509fa1eace0150d945d49Fd,0x0000000000000000000000000000000000000000,2000000000000000,3.74,3734913250000000000,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,EthStaker,APPROVED,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,Ethereum Infrastructure,NaN,0
81977,0xedd02d1a9ce95dee30d60354b59505399a40a380db7b...,0x05a0ebe8868ae8e3a40deed82938643d1c7c192be958...,17125925,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,8,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,0x30C7F4F7504D6366916f669cd8E731ED4dF6C702,0xD165df4296C85e780509fa1eace0150d945d49Fd,0x6B175474E89094C44Da98b954EedeAC495271d0F,5010000000000000000,5.01,5010000000000000000,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,EthStaker,APPROVED,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,Ethereum Infrastructure,NaN,0
82001,0x020f9ba0efc2944f5c9be10c9971b50defdc3f049337...,0x22ad373d0a0438d397bc63c9326df969b6467e289571...,17126810,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,8,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,0x179A862703a4adfb29896552DF9e307980D19285,0xD165df4296C85e780509fa1eace0150d945d49Fd,0x0000000000000000000000000000000000000000,5000000000000000,9.33,9333447300000000000,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,EthStaker,APPROVED,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,Ethereum Infrastructure,NaN,0
82008,0x6007a113c195395952665de4d0110f526cae62cf04fe...,0x9487e8a7772d1c187fecb0c7dd7003a7ee9d95778cdb...,17127434,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,8,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,0xfc04F9E8f7410198639135F8e02660e55Bf50d19,0xD165df4296C85e780509fa1eace0150d945d49Fd,0x0000000000000000000000000000000000000000,800000000000000,1.50,1501746640000000000,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,EthStaker,APPROVED,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,Ethereum Infrastructure,NaN,0
82018,0x83b8edce33892a95f6733885dfaa97274aea175abe4a...,0x69bdb0cddec08e21f47d986923ea7cfcb3313ff47a1f...,17127462,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,8,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,0xA2b16c27c0766A1Df18892F7b0413b4f5806ee4D,0xD165df4296C85e780509fa1eace0150d945d49Fd,0x6B175474E89094C44Da98b954EedeAC495271d0F,1000000000000000000,1.00,1000000000000000000,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,EthStaker,APPROVED,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,Ethereum Infrastructure,NaN,0
82031,0x5d31b5e03314e196bd098f8cac61a6ae905e1cb018cd...,0x81447c8e19a0d6a6976ebbbed2096a8d930217b85ab6...,17127637,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,8,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,0x54C1530628A67c3eF537f01712b9Bb517aCDAeC2,0xD165df4296C85e780509fa1eace0150d945d49Fd,0x0000000000000000000000000000000000000000,10000000000000000,18.95,18931471500000000000,0x76eabce511626d1d54bdae4770c3dc172bffca39ba53...,EthStaker,APPROVED,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,Ethereum Infrastructure,NaN,0
82050,0x547647a31b8cc174a5dd888b4ea025df005ca798fa8b...,0x43528acd41dc8ce2231756ff36fbdabee25da53d6b18...,17127864,0x76eabce511626d1d54bdae47

In [5]:
# read self_votes.csv
df = pd.read_csv('self_votes.csv')
print(df['amountUSD'].sum())

5231.73080399


In [19]:
self_votes = self_votes.sort_values('amountUSD', ascending=False).head(15)
self_votes['title'] = self_votes['title'].str[:15]
fig = px.bar(self_votes, x='title', y='amountUSD', color='title', title='Self Votes')
fig.update_traces(texttemplate='%{y:$.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=7, uniformtext_mode='hide')
fig.show()

In [37]:
# TEMP CLIMATE PULL

df_alpha_climate = load_round_votes_data('0x1b165fE4DA6bC58AB8370DDC763d367D29F50Ef0')
df_alpha_climate_voters = df_alpha_climate['voter'].unique()
df_alpha_climate_voters = pd.DataFrame(df_alpha_climate_voters, columns=['address'])
display(df_alpha_climate_voters)
# filter to round_id = 0x421510312C40486965767be5Ea603Aa8a5707983
df_climate = df_all[df_all['round_id'] == '0x421510312C40486965767be5Ea603Aa8a5707983']
df_climate_voters = df_climate['voter'].unique()
df_climate_voters = pd.DataFrame(df_climate_voters, columns=['address'])
# save to csv
# append df_climate_voters to df_alpha_climate_voters
df_climate_voters = pd.concat([df_alpha_climate_voters, df_climate_voters])
# drop duplicates
df_climate_voters = df_climate_voters.drop_duplicates()
df_climate_voters.to_csv('alpha_beta_climate_voters.csv', index=False)


In [162]:
def load_chain_data(chain_id):
    chain_url = 'https://indexer-grants-stack.gitcoin.co/data/' + chain_id + '/rounds.json'
    try:
        response = requests.get(chain_url)
        if response.status_code == 200:
            chain_data = response.json()
            rounds = []
            for round in chain_data:
                if round['metadata'] is not None:
                    round_data = {
                        'round_id': round['id'],
                        'name': round['metadata']['name'],
                        'amountUSD': round['amountUSD'],
                        'votes': round['votes'],
                        'description': round['metadata']['description'] if 'description' in round['metadata'] else '',
                        'matchingFundsAvailable': round['metadata']['matchingFunds']['matchingFundsAvailable'] if 'matchingFunds' in round['metadata'] else '',
                        'matchingCap': round['metadata']['matchingFunds']['matchingCap'] if 'matchingFunds' in round['metadata'] else '',
                        'roundStartTime': datetime.datetime.utcfromtimestamp(int(round['roundStartTime'])), # create a datetime object from the timestamp in UTC time
                        'roundEndTime': datetime.datetime.utcfromtimestamp(int(round['roundEndTime']))
                    }
                    rounds.append(round_data)
            df = pd.DataFrame(rounds)
            # Filter to beta rounds
            start_time = datetime.datetime(2023, 4, 26, 15, 0, 0)
            end_time = datetime.datetime(2023, 5, 9, 23, 59, 0)
            # filter to only include rounds with votes > 0 and roundStartTime <= start_time and roundEndTime == end_time
            df = df[(df['votes'] > 0) & (df['roundStartTime'] <= start_time) & (df['roundEndTime'] == end_time)]
            return df 
    except: 
        return pd.DataFrame()

In [163]:

# display all
pd.set_option('display.max_rows', None)
print('boobs')
display(load_chain_data('1'))

boobs


,round_id,name,amountUSD,votes,description,matchingFundsAvailable,matchingCap,roundStartTime,roundEndTime
12,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,Web3 Open Source Software Round,329305.79,81508,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
13,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,Ethereum Infrastructure,41806.14,4287,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
15,0x274554EB289004e15A7679123901B7F070dDa0fa,ZK Tech Round,30570.96,8498,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
16,0xAA40E2E5c8df03d792A52B5458959C320F86ca18,Web3 Community and Education,62002.91,2835,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
17,0x421510312C40486965767be5Ea603Aa8a5707983,Climate Solutions,39944.54,1894,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
18,0x8aA06b3b8cAc2970857F4E0fD78F21dc01AAdE94,Metacrisis,17534.03,591,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
19,0x905eFBaBe2d52cD648FAdfafCeC8D6C8c60F7423,The Phantom Menace,1405.05,183,,,,2023-04-25 00:00:00,2023-05-09 23:59:00
20,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,Token Engineering,6112.97,254,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
21,0x9C3B81967EafBA0a451E324417DD4F3F353b997b,Web3 Social,25294.66,2519,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
22,0xF1C021DF6Dc6b2dC2E5A837CDFDDC2f42503233B,Mantle Grants 1,9296.43,1326,,,,2023-04-25 12:00:00,2023-05-09 23:59:00
